# LLM agent Hackathon 
## Alireza Ghafarollahi, MIT

# AG2: Building LLM-powered multi-agent systems

In [ ]:
!pip install ag2
!pip install openai

In [3]:
import json
import os

with open('config.json') as f:
    config = json.load(f)

OPENAI_API_KEY = config["api_key"]
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Example: Group chat to assist the user with a tech-related query

## Agents

- ## coordinator agent

- ## technical support agent

- ## general support agent

## Tools

- ## Classify_query

# Enhance our system with context variables:

- ## Create initial context variables with session information
- ## Track and store each user query in the context
- ## Record solutions provided by technical support
- ## Intentionally print the context variables before and after processing in the functions for debugging

In [4]:
from typing import Annotated
from datetime import datetime
from autogen import ConversableAgent, LLMConfig
from autogen.agentchat import initiate_group_chat
from autogen.agentchat.group.patterns import AutoPattern
from autogen.agentchat.group import ReplyResult, AgentNameTarget, ContextVariables

# Initialize context variables

In [5]:
support_context = ContextVariables(data={
    "session_start": datetime.now().isoformat(),
    "query_history": [],
    "solutions_provided": [],
    "query_count": 0,
    "solution_count": 0
})

# Define tools that use context variables

In [6]:
def classify_and_log_query(
    query: Annotated[str, "The user query to classify"],
    context_variables: ContextVariables
) -> ReplyResult:
    """Classify a user query as technical or general and log it in the context."""

    # Printing the context variables
    print(f"{context_variables.to_dict()=}")

    # Record this query in history
    query_history = context_variables.get("query_history", [])
    query_record = {
        "timestamp": datetime.now().isoformat(),
        "query": query
    }
    query_history.append(query_record)
    context_variables["query_history"] = query_history
    context_variables["last_query"] = query
    context_variables["query_count"] = len(query_history)

    # Basic classification logic
    technical_keywords = ["error", "bug", "broken", "crash", "not working", "shutting down"]
    is_technical = any(keyword in query.lower() for keyword in technical_keywords)

    # Update context with classification
    if is_technical:
        target_agent = AgentNameTarget("tech_agent")
        context_variables["query_type"] = "technical"
        message = "This appears to be a technical issue. Routing to technical support..."
    else:
        target_agent = AgentNameTarget("general_agent")
        context_variables["query_type"] = "general"
        message = "This appears to be a general question. Routing to general support..."

    # Printing the context variables
    print(f"{context_variables.to_dict()=}")

    return ReplyResult(
        message=message,
        target=target_agent,
        context_variables=context_variables
    )

def provide_technical_solution(
    solution: Annotated[str, "Technical solution to provide"],
    context_variables: ContextVariables
) -> ReplyResult:
    """Provide a technical solution and record it in the context."""

    # Printing the context variables
    print(f"{context_variables.to_dict()=}")

    # Record the solution
    last_query = context_variables.get("last_query", "your issue")
    solutions_provided = context_variables.get("solutions_provided", [])

    solution_record = {
        "timestamp": datetime.now().isoformat(),
        "query": last_query,
        "solution": solution
    }
    solutions_provided.append(solution_record)

    # Update context
    context_variables["solutions_provided"] = solutions_provided
    context_variables["last_solution"] = solution
    context_variables["solution_count"] = len(solutions_provided)

    # Printing the context variables
    print(f"{context_variables.to_dict()=}")

    return ReplyResult(
        message=solution,
        context_variables=context_variables
    )

# Create the agents

In [8]:
llm_config = LLMConfig(api_type="openai", model="gpt-4o-mini")

with llm_config:
    coordinator_agent = ConversableAgent(
        name="coordinator_agent",
        system_message="""You are a coordinator agent. For each user query,
        identify whether it is a technical issue or a general question.
        Use the classify_and_log_query tool to categorize and log queries.""",
        functions=[classify_and_log_query]
    )

    tech_agent = ConversableAgent(
        name="tech_agent",
        system_message="""You solve technical problems like software bugs
        and hardware issues. After analyzing the problem, use the provide_technical_solution
        tool to format your response consistently and log it for future reference.

        Check context variables for any user history before responding.""",
        functions=[provide_technical_solution]
    )

    general_agent = ConversableAgent(
        name="general_agent",
        system_message="""You handle general, non-technical support questions.
        Check context variables for user history before responding to provide
        a personalized experience."""
    )

# User agent
user = ConversableAgent(name="user", human_input_mode="ALWAYS")

# Set up the conversation pattern with context variables

In [9]:
pattern = AutoPattern(
    initial_agent=coordinator_agent,
    agents=[coordinator_agent, tech_agent, general_agent],
    user_agent=user,
    context_variables=support_context,  # Pass our initialized context
    group_manager_args={"llm_config": llm_config}
)

# Run the chat

In [10]:
result, final_context, last_agent = initiate_group_chat(
    pattern=pattern,
    messages="My laptop keeps shutting down randomly. Can you help?",
    max_rounds=10
)

user (to chat_manager):

My laptop keeps shutting down randomly. Can you help?

--------------------------------------------------------------------------------

Next speaker: coordinator_agent


>>>>>>>> USING AUTO REPLY...
coordinator_agent (to chat_manager):

***** Suggested tool call (call_tgmeap2YCWq4G5lz0SdFAKAl): classify_and_log_query *****
Arguments: 
{"query":"My laptop keeps shutting down randomly. Can you help?"}
***************************************************************************************

--------------------------------------------------------------------------------

Next speaker: _Group_Tool_Executor


>>>>>>>> EXECUTING FUNCTION classify_and_log_query...
Call ID: call_tgmeap2YCWq4G5lz0SdFAKAl
Input arguments: {'query': 'My laptop keeps shutting down randomly. Can you help?'}
context_variables.to_dict()={'session_start': '2025-05-26T23:33:59.464579', 'query_history': [{'timestamp': '2025-05-26T23:34:17.998150', 'query': 'My laptop keeps shutting down randomly

Replying as user. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit



>>>>>>>> TERMINATING RUN (898bf5be-0590-40a3-8a50-0fd72ce9687a): User requested to end the conversation

>>>>>>>> TERMINATING RUN (d803ed87-cf75-4308-960d-6599274b32be): No reply generated


In [12]:
support_context.to_dict()

{'session_start': '2025-05-26T23:33:59.464579',
 'query_history': [{'timestamp': '2025-05-26T23:34:17.998150',
   'query': 'My laptop keeps shutting down randomly. Can you help?'},
  {'timestamp': '2025-05-26T23:35:07.504984',
   'query': 'My laptop keeps shutting down randomly. Can you help?'}],
 'solutions_provided': [{'timestamp': '2025-05-26T23:35:11.871700',
   'query': 'My laptop keeps shutting down randomly. Can you help?',
   'solution': 'If your laptop keeps shutting down randomly, it could be due to several reasons: overheating, a failing battery, faulty hardware, or software issues. Here are steps you can take to troubleshoot:\n\n1. **Check for Overheating:** Ensure that the laptop vents are not blocked and that it is not overheating. Clean any dust from the fans and vents.\n\n2. **Battery Health:** Check the battery health status in your system settings and consider performing a battery calibration. If the battery is old or damaged, it might need replacement.\n\n3. **Hardwa